In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


In [ ]:
import os 
import src.globals as g
import src.utils as utils

import src.data_handler as handling

In [ ]:
dataset_path = os.path.join(g.DATA_FOLDER,'training_set.json')
squad_dataset = handling.RawSquadDataset(dataset_path)

df = squad_dataset.train_df.copy()

df.head()

In [ ]:
model, vocab = utils.get_Glove_model_and_vocab()

In [ ]:
from tokenizers import  Tokenizer, Encoding
from tokenizers.models import WordLevel
from tokenizers.normalizers import Lowercase, Strip, StripAccents, NFD, BertNormalizer
from tokenizers.normalizers import Sequence as NormSequence
from tokenizers.pre_tokenizers import Punctuation, Whitespace
from tokenizers.pre_tokenizers import Sequence as PreSequence
from tokenizers.processors import TemplateProcessing
from tokenizers.trainers import WordLevelTrainer


tokenizer = Tokenizer(WordLevel(unk_token=g.UNK_TOKEN))
tokenizer.normalizer = BertNormalizer(handle_chinese_chars=False) #NormSequence([NFD(), StripAccents(), Lowercase(), Strip()])    
tokenizer.pre_tokenizer = PreSequence([Whitespace(), Punctuation()])

tokenizer.post_processor = TemplateProcessing(
    single="[SOS] $A [EOS]",
    pair="[SOS] $A [EOS] [SOS]:1 $B:1 [EOS]:1",
    special_tokens=[
        ("[SOS]", 2),
        ("[EOS]", 3),
    ],
)


trainer = WordLevelTrainer(special_tokens=[g.PAD_TOKEN,g.UNK_TOKEN,g.SOS_TOKEN,g.EOS_TOKEN],vocab_size=65000)   #min_frequency

l = df.context.to_list() + df.answer.to_list()
#l = df.question.to_list()
tokenizer.train_from_iterator(l,trainer=trainer) 
tokenizer.enable_padding(direction="right", pad_id=tokenizer.token_to_id(g.PAD_TOKEN), pad_type_id=1, pad_token=g.PAD_TOKEN)


In [ ]:
tokenizer.get_vocab_size()
len(tokenizer.get_vocab())

In [ ]:
tokenizer.add_special_tokens([g.PAD_TOKEN,g.UNK_TOKEN]) #,g.SOS_TOKEN,g.EOS_TOKEN

In [ ]:
# l = df.context.to_list() + df.answer.to_list() 
s = set()
for e in l :
    # if 'intellectu' in e:
    #     print(e)
    s.update(e.split())

len(s)
        

#tokenizer.encode('To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?').tokens



In [ ]:
tokenizer.save('data/tokenizer.json')

In [ ]:
n = 0
for e in tokenizer.get_vocab().keys() :
    if e not in vocab:
        # print(e)
        n+=1
n

In [ ]:
tokenizer.get_vocab()["tδ"]

df[df['context'].str.contains('tδ')]

for e in l :
    if 'tδ' in e:
        print(e)

In [ ]:
tokenizer.id_to_token(2)
tokenizer.token_to_id('plda')

In [ ]:
df[df['question_id']=='5726d73d708984140094d310']['context']

In [ ]:
r1 = df.loc[49591]
r2 = df.loc[49593]
s1 = r1['context']
s2 = r2['context']
s1
s2

In [ ]:
starts = [r1['label_char'][0],r2['label_char'][0]]
ends = [r1['label_char'][1],r2['label_char'][1]]

starts
ends

In [ ]:
r1['answer']
r2['answer']

In [ ]:
encodings: list[Encoding] = tokenizer.encode_batch([s1,s2])

print([e.ids for e in encodings])
print([e.attention_mask for e in encodings])
# print([e.offsets for e in encodings])
print([e.char_to_token(starts[i]) for i,e in enumerate(encodings)])
print([e.char_to_token(ends[i]-1) for i,e in enumerate(encodings)])
print([e.type_ids for e in encodings])
print([e.tokens for e in encodings])
print([e.special_tokens_mask for e in encodings])

print(encodings[0].tokens[94:100])

In [ ]:
print(tokenizer.get_vocab()['hokkien'])

In [ ]:
import gensim.downloader as gloader
from gensim.models import KeyedVectors
import time 
import pandas as pd 
import numpy as np

import logging 

logger = logging.getLogger(g.LOG_NAME)

In [ ]:
model['hello'].shape
type(np.float32)

In [ ]:
a = np.array([1,2,3,4,5])
np.concatenate([a,[0,0,0]])


In [2]:
import torch
from torch import nn
import numpy as np

In [ ]:
a = torch.tensor([[[1,2,3],[1,2,3],[1,2,3]],[[4,5,6],[4,5,6],[4,5,6]],[[7,8,9],[7,8,9],[7,8,9]]])

b = torch.tensor([[[1,2,3,1],[1,2,3,1],[1,2,3,0]],[[4,5,6,0],[4,5,6,1],[4,5,6,0]],[[7,8,9,1],[7,8,9,1],[7,8,9,1]]])


start = np.array([0,1,0])
end = np.array([1,1,2])

c = (start[:,None] <= np.arange(a.shape[1])).view('i1')    #np.less_equal.outer(start, np.arange(a.shape[1])).view('i1')
d = (end[:,None] >= np.arange(a.shape[1])).view('i1')
c
d
f = c*d

f

f = torch.from_numpy(f)

f = f.unsqueeze(-1)

r = torch.cat((a,f),dim=2)


In [2]:
enc_m = torch.rand((20,5))
enc_m[0] = torch.zeros(5)

enc_emb = nn.Embedding.from_pretrained(enc_m,padding_idx=0)

h_dim = 4

rnn = nn.LSTM(5+1, h_dim, batch_first=True, bidirectional=True)

In [3]:
ctx_ids = torch.tensor([[1,2,3,0,0,0],[3,7,8,12,17,19],[3,15,4,1,0,0]])

In [4]:
ctx_embeds = enc_emb(ctx_ids)

ctx_embeds

tensor([[[0.0342, 0.2083, 0.5885, 0.5444, 0.6211],
         [0.2240, 0.0550, 0.3989, 0.2854, 0.8079],
         [0.0920, 0.3813, 0.9056, 0.2165, 0.2945],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.0920, 0.3813, 0.9056, 0.2165, 0.2945],
         [0.1063, 0.5565, 0.2262, 0.9885, 0.4189],
         [0.2214, 0.5502, 0.4919, 0.6034, 0.9868],
         [0.1193, 0.6492, 0.9727, 0.9969, 0.1712],
         [0.5893, 0.3778, 0.3170, 0.4766, 0.7206],
         [0.3725, 0.2737, 0.1332, 0.8680, 0.7826]],

        [[0.0920, 0.3813, 0.9056, 0.2165, 0.2945],
         [0.1832, 0.6558, 0.6514, 0.7944, 0.9617],
         [0.9532, 0.8505, 0.3287, 0.4101, 0.2038],
         [0.0342, 0.2083, 0.5885, 0.5444, 0.6211],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]]])

In [5]:
start = torch.tensor([0,2,3])
end = torch.tensor([1,4,3])


t1 = torch.le(start.unsqueeze(-1),torch.arange(ctx_embeds.shape[1])).float()
t2 = torch.ge(end.unsqueeze(-1),torch.arange(ctx_embeds.shape[1])).float()


m = torch.mul(t1,t2).unsqueeze(-1)

r = torch.cat((ctx_embeds,m),dim=2)

r




tensor([[[0.0342, 0.2083, 0.5885, 0.5444, 0.6211, 1.0000],
         [0.2240, 0.0550, 0.3989, 0.2854, 0.8079, 1.0000],
         [0.0920, 0.3813, 0.9056, 0.2165, 0.2945, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.0920, 0.3813, 0.9056, 0.2165, 0.2945, 0.0000],
         [0.1063, 0.5565, 0.2262, 0.9885, 0.4189, 0.0000],
         [0.2214, 0.5502, 0.4919, 0.6034, 0.9868, 1.0000],
         [0.1193, 0.6492, 0.9727, 0.9969, 0.1712, 1.0000],
         [0.5893, 0.3778, 0.3170, 0.4766, 0.7206, 1.0000],
         [0.3725, 0.2737, 0.1332, 0.8680, 0.7826, 0.0000]],

        [[0.0920, 0.3813, 0.9056, 0.2165, 0.2945, 0.0000],
         [0.1832, 0.6558, 0.6514, 0.7944, 0.9617, 0.0000],
         [0.9532, 0.8505, 0.3287, 0.4101, 0.2038, 0.0000],
         [0.0342, 0.2083, 0.5885, 0.5444, 0.6211, 1.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.

In [6]:
out, _ = rnn(r)

print(out)

tensor([[[-0.2022, -0.1078,  0.0982, -0.0565,  0.0529, -0.1729, -0.0632,
          -0.1712],
         [-0.2916, -0.1140,  0.1835, -0.0572,  0.0771, -0.1673, -0.0624,
          -0.1728],
         [-0.3321, -0.1095,  0.2821, -0.0503,  0.1080, -0.1110,  0.0575,
          -0.1187],
         [-0.2702, -0.1124,  0.2768, -0.0088,  0.2418, -0.1393,  0.0445,
          -0.1359],
         [-0.2657, -0.1125,  0.2719,  0.0141,  0.2193, -0.1282,  0.0416,
          -0.1142],
         [-0.2610, -0.1121,  0.2690,  0.0239,  0.1607, -0.0981,  0.0300,
          -0.0746]],

        [[-0.1948, -0.0824,  0.1723, -0.0545,  0.0888, -0.1376,  0.0804,
          -0.0609],
         [-0.2262, -0.0842,  0.3153, -0.0096,  0.1223, -0.1930,  0.0737,
          -0.0603],
         [-0.3226, -0.1003,  0.3598, -0.0157,  0.0318, -0.1474, -0.0390,
          -0.0838],
         [-0.3730, -0.1423,  0.3673, -0.0218,  0.0724, -0.1046,  0.0315,
           0.0210],
         [-0.3900, -0.1090,  0.4555,  0.0242,  0.1192, -0.1807, -0.0

In [11]:
out.shape
v = torch.mean(out,dim=1)
v.shape
v = v.unsqueeze(1).expand(out.size())

torch.add(v,out)

torch.Size([3, 6, 8])

torch.Size([3, 8])

tensor([[[-4.7264e-01, -2.1914e-01,  3.2841e-01, -7.9039e-02,  1.9619e-01,
          -3.0905e-01, -5.5260e-02, -3.0239e-01],
         [-5.6204e-01, -2.2537e-01,  4.1372e-01, -7.9720e-02,  2.2043e-01,
          -3.0340e-01, -5.4385e-02, -3.0400e-01],
         [-6.0256e-01, -2.2093e-01,  5.1239e-01, -7.2817e-02,  2.5131e-01,
          -2.4718e-01,  6.5463e-02, -2.4992e-01],
         [-5.4063e-01, -2.2383e-01,  5.0708e-01, -3.1315e-02,  3.8510e-01,
          -2.7544e-01,  5.2449e-02, -2.6716e-01],
         [-5.3616e-01, -2.2388e-01,  5.0219e-01, -8.3749e-03,  3.6259e-01,
          -2.6428e-01,  4.9616e-02, -2.4544e-01],
         [-5.3144e-01, -2.2346e-01,  4.9923e-01,  1.3773e-03,  3.0402e-01,
          -2.3419e-01,  3.7963e-02, -2.0588e-01]],

        [[-4.9615e-01, -1.7703e-01,  5.2979e-01, -5.4246e-02,  1.8101e-01,
          -2.9813e-01,  1.0448e-01, -1.1465e-01],
         [-5.2755e-01, -1.7875e-01,  6.7285e-01, -9.2843e-03,  2.1446e-01,
          -3.5345e-01,  9.7764e-02, -1.1398e-01]

In [ ]:
torch.add(out,v)

In [11]:
b = torch.zeros((3,5,10))

c = torch.rand(3,10)

b.shape
c.shape

torch.Size([3, 5, 10])

torch.Size([3, 10])

In [12]:
b
c

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])

tensor([[0.3278, 0.2552, 0.4009, 0.8780, 0.1698, 0.7730, 0.6941, 0.6092, 0.2546,
         0.3166],
        [0.4466, 0.7695, 0.3473, 0.1396, 0.9413, 0.7366, 0.5315, 0.9089, 0.8274,
         0.3054],
        [0.1927, 0.0131, 0.3705, 0.5231, 0.0566, 0.2250, 0.9373, 0.5516, 0.4360,
         0.7231]])

In [15]:
b[:,1,:] = c

b

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.3278, 0.2552, 0.4009, 0.8780, 0.1698, 0.7730, 0.6941, 0.6092,
          0.2546, 0.3166],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.4466, 0.7695, 0.3473, 0.1396, 0.9413, 0.7366, 0.5315, 0.9089,
          0.8274, 0.3054],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.000

In [ ]:
answ_ids = torch.tensor([[1,2,0],[8,12,17],[1,0,0]]) 

In [ ]:
answ_embeds = enc_emb(answ_ids)

answ_embeds

In [ ]:
z = torch.zeros(answ_embeds.shape[0],answ_embeds.shape[1],h_dim*2)
z

In [ ]:

for i in range(answ_embeds.shape[0]):
    z[i,0:end[i]+1-start[i],:] = out[i,start[i]:end[i]+1,:]

z


In [ ]:
torch.cat((z,answ_embeds),dim=2)

In [ ]:
answ_embeds

In [ ]:
a = torch.tensor([[[1,2,3],[11,2,3],[1,2,3]],[[4,5,6],[43,5,6],[4,5,6]]])
b = torch.tensor([[[2,3,4],[2,3,4],[2,3,4]],[[5,6,7],[5,6,7],[5,6,7]]])

c = (a,b)

tuple((torch.cat((hidden[0:hidden.size(0):2], hidden[1:hidden.size(0):2]), dim=2) for hidden in c))
